This notebook performs the preprocessing of the raw data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

from src.preprocess import parse_all_data, clean_raw_data, computeTimeDeltaTraces

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "none"

import src.colors as colors
from src.helpers import Modes, Parameters

from src.stats import ThompsonCI_twosided

In [ ]:
# Clean up and bug correction of the data collected by Anna
# clean_raw_data()

In [25]:
# Create and save the median values for faster plotting of the different settings
computeTimeDeltaTraces()

Processed data retrieved.
Done with TimeDeltaTraces_A_0_(-35).csv
Done with TimeDeltaTraces_A_0_(-34).csv
Done with TimeDeltaTraces_A_0_(-33).csv
Done with TimeDeltaTraces_A_0_(-32).csv
Done with TimeDeltaTraces_A_0_(-16).csv
Done with TimeDeltaTraces_A_0_(-19).csv
Done with TimeDeltaTraces_A_0_(-23).csv
Done with TimeDeltaTraces_A_0_(-29).csv
Done with TimeDeltaTraces_A_0_(-30).csv
Done with TimeDeltaTraces_A_0_(-31).csv
Done with TimeDeltaTraces_A_0_(-14).csv
Done with TimeDeltaTraces_A_0_(-11).csv
Done with TimeDeltaTraces_A_0_(-7).csv
Done with TimeDeltaTraces_A_0_(-4).csv
Done with TimeDeltaTraces_A_0_(-3).csv
Done with TimeDeltaTraces_A_0_(-2).csv
Done with TimeDeltaTraces_A_0_(-1).csv
Done with TimeDeltaTraces_A_0_(0).csv
Done with TimeDeltaTraces_A_0_(1).csv
Done with TimeDeltaTraces_A_0_(16).csv
Done with TimeDeltaTraces_A_0_(13).csv
Done with TimeDeltaTraces_A_0_(9).csv
Done with TimeDeltaTraces_A_0_(5).csv
Done with TimeDeltaTraces_A_0_(4).csv
Done with TimeDeltaTraces_A_0_(

Done with TimeDeltaTraces_B_1_(13).csv
Done with TimeDeltaTraces_B_1_(9).csv
Done with TimeDeltaTraces_B_1_(5).csv
Done with TimeDeltaTraces_B_1_(4).csv
Done with TimeDeltaTraces_B_1_(3).csv
Done with TimeDeltaTraces_B_1_(2).csv
Done with TimeDeltaTraces_B_1_(-10).csv
Done with TimeDeltaTraces_B_1_(14).csv
Done with TimeDeltaTraces_B_1_(11).csv
Done with TimeDeltaTraces_B_1_(8).csv
Done with TimeDeltaTraces_B_1_(-8).csv
Done with TimeDeltaTraces_B_1_(15).csv
Done with TimeDeltaTraces_B_1_(12).csv
Done with TimeDeltaTraces_B_1_(19).csv
Done with TimeDeltaTraces_B_1_(6).csv
Done with TimeDeltaTraces_B_1_(-13).csv
Done with TimeDeltaTraces_B_1_(-9).csv
Done with TimeDeltaTraces_B_1_(-5).csv
Done with TimeDeltaTraces_B_1_(7).csv
Done with TimeDeltaTraces_B_1_(-12).csv
Done with TimeDeltaTraces_B_1_(-15).csv
Done with TimeDeltaTraces_B_1_(-6).csv
Done with TimeDeltaTraces_B_1_(20).csv
Done with TimeDeltaTraces_B_1_(17).csv
Done with TimeDeltaTraces_B_1_(10).csv
Done with TimeDeltaTraces_B_1

In [6]:
n_samples = 10
percentile = 50
confidence = 75
ThompsonCI_twosided(n_samples, percentile, confidence)

(2, 7)

In [ ]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data(force_computation='prr')

In [ ]:
a = df.PowerDelta.dropna().unique().tolist()
# print(sorted(a))

b = range(-16,17)
print([x for x in b])

Processed data retrieved.


In [32]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data()
data_path = Path('data_preprocessed')

PowerDelta,SamePayload,TransPair = 9, 1, "B"

filter = (
#     (df['DateTime'] == "2019-10-04 13:55:00") & 
    (df['TransPair'] == "B") & 
    (df['SamePayload'] == 1) & 
    (df["PowerDelta"] == 9)  
)
filtered_df = df.where(filter).dropna()
# display(filtered_df)

Processed data retrieved.


In [60]:
figure = go.Figure()


# Load the median and CI data
file_path = data_path / Parameters['TransPair'][TransPair]['path'] / Parameters['SamePayload'][SamePayload]['path']
file_name = 'TimeDeltaTraces_%s_%s_(%i).csv' % (TransPair,SamePayload,PowerDelta)
df_median = pd.read_csv(file_path / file_name)

for mode in Modes:
    
    mode_filter = (filtered_df["Mode"] == Modes[mode]['id'])
    mode_df = filtered_df.where(mode_filter).dropna()
    
    # Force displaying the trace, even if empty
    if len(mode_df) == 0:
        x_data = [np.nan]
        y_data = [np.nan]
        x_median = [np.nan]
        y_median = [np.nan]
    else: 
        x_data = mode_df["TimeDelta"].to_list()
        y_data = mode_df["PRR"].to_list()
        
        # Extract median data
        x_median = df_median["TimeDelta"].to_list()
        y_median = df_median['median_'+mode].to_list()

        # Extract CI data
        y_LB = df_median['LB_'+mode].to_list()
        y_UB = df_median['UB_'+mode].to_list()
        
        # Prepare the trace for the CI area
        x_CI = x_median + x_median[::-1]
        y_CI = y_LB + y_UB[::-1]
    
    scatter = go.Scatter(
        x=x_data,
        y=y_data,
        mode='markers',
        marker={'color':Modes[mode]['color']},
        showlegend=True,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label'],
        opacity=0.7,
        hoverinfo='skip',
    )
    figure.add_trace(scatter)
    
    
    median_line = go.Scatter(
        x=x_median,
        y=y_median,
        mode='lines',
        line={'color':Modes[mode]['color']},
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' median',
    )
    figure.add_trace(median_line)
    
    CI = go.Scatter(
        x=x_CI,
        y=y_CI,
        mode='lines',
        line={
            'color':Modes[mode]['color'],
            'width':0
        },
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' CI',
        fill='toself',
        hoverinfo='skip',
        opacity=0.3,
    )
    figure.add_trace(CI)
    
    
    
custom_layout = {
    "title":{
        'text':'PRR = f(Time Delta) <br> with Power Delta = %d dB' % 0,
    },
#     "width":linewidth_px,
#     "height":220,
#     "font":{"size":font_size},
    "margin":dict(t=150),
    "xaxis":{
        'title':{'text':'Transmitters Time Delta [ticks]'},
            },
    "yaxis":{
        'range':[0,103]
    },
    "legend":{
#         'font':{'size':font_size},
        'orientation':"h",
        'x': 0.5,
        'y':1.15,
        'xanchor':"center",
        'yanchor':"top",
    }
}
figure.update_layout(custom_layout)
figure.show()